In [1]:
import os
import pprint as pp

import geopandas as gpd
import teotil3 as teo

In [2]:
# Datasets of interest
reg_year = 2022
admin_year = 2022
data_fold = f"/home/jovyan/shared/teotil3/core_data_june_{reg_year}"
teo_gpkg = os.path.join(data_fold, "tidied", f"teotil3_data.gpkg")
reg_gdf = gpd.read_file(
    teo_gpkg, layer=f"regine_{reg_year}_admin_{admin_year}", driver="GPKG"
)
reg_gdf.head()

,regine,a_cat_land_km2,a_cat_poly_km2,upstr_a_km2,upstr_runoff_Mm3/yr,q_sp_m3/s/km2,runoff_mm/yr,q_cat_m3/s,vassom,komnr,...,a_glacier_km2,a_lake_km2,a_other_km2,a_sea_km2,a_upland_km2,a_urban_km2,a_wood_km2,ar50_tot_a_km2,a_lake_nve_km2,geometry
0,001.10,1.16,1.442790,0.00,0.00,0.01393,439.597368,0.016159,001,3001,...,0.0,0.000013,3.116485e-01,0.281940,0.000000,0.000000,0.849188,0.849201,0.000000,"MULTIPOLYGON (((297006.830 6543966.950, 297169..."
1,001.1A1,1.43,1.432479,777.90,448.15,0.01419,447.802344,0.020292,001,3001,...,0.0,0.048098,2.223036e-03,0.000067,0.004615,0.000000,1.377476,1.430189,0.043955,"MULTIPOLYGON (((297505.440 6543157.790, 297543..."
2,001.1A20,0.34,0.340160,777.90,448.15,0.01036,326.936736,0.003522,001,3001,...,0.0,0.036623,0.000000e+00,0.000045,0.000000,0.000000,0.303492,0.340114,0.000000,"MULTIPOLYGON (((297770.368 6543429.036, 297787..."
3,001.1A2A,17.65,17.647822,58.96,22.97,0.01210,381.846960,0.213565,001,3001,...,0.0,0.122567,0.000000e+00,0.000000,0.467374,0.131585,15.030746,17.647822,0.186340,"MULTIPOLYGON (((299678.370 6544460.320, 299667..."
4,001.1A2B,41.30,41.298255,41.30,16.23,0.01245,392.892120,0.514185,001,3001,...,0.0,7.513687,1.421085e-14,0.000000,2.250799,0.161524,29.798394,41.298255,7.344123,"MULTIPOLYGON (((303353.460 6552989.330, 303341..."


In [3]:
# Determine hydrological connectivity
reg_gdf = teo.io.assign_regine_hierarchy(
    reg_gdf,
    regine_col="regine",
    regine_down_col="regine_down",
    order_coastal=False,
    nan_to_vass=True,
    land_to_vass=True,
    add_offshore=True,
)
reg_gdf.head()

100.00 % of regines assigned.


,regine,regine_down,a_cat_land_km2,a_cat_poly_km2,upstr_a_km2,upstr_runoff_Mm3/yr,q_sp_m3/s/km2,runoff_mm/yr,q_cat_m3/s,vassom,...,a_glacier_km2,a_lake_km2,a_other_km2,a_sea_km2,a_upland_km2,a_urban_km2,a_wood_km2,ar50_tot_a_km2,a_lake_nve_km2,geometry
0,001.10,001.,1.16,1.442790,0.00,0.00,0.01393,439.597368,0.016159,001,...,0.0,0.000013,3.116485e-01,0.281940,0.000000,0.000000,0.849188,0.849201,0.000000,"MULTIPOLYGON (((297006.830 6543966.950, 297169..."
1,001.1A1,001.,1.43,1.432479,777.90,448.15,0.01419,447.802344,0.020292,001,...,0.0,0.048098,2.223036e-03,0.000067,0.004615,0.000000,1.377476,1.430189,0.043955,"MULTIPOLYGON (((297505.440 6543157.790, 297543..."
2,001.1A20,001.1A1,0.34,0.340160,777.90,448.15,0.01036,326.936736,0.003522,001,...,0.0,0.036623,0.000000e+00,0.000045,0.000000,0.000000,0.303492,0.340114,0.000000,"MULTIPOLYGON (((297770.368 6543429.036, 297787..."
3,001.1A2A,001.1A20,17.65,17.647822,58.96,22.97,0.01210,381.846960,0.213565,001,...,0.0,0.122567,0.000000e+00,0.000000,0.467374,0.131585,15.030746,17.647822,0.186340,"MULTIPOLYGON (((299678.370 6544460.320, 299667..."
4,001.1A2B,001.1A2A,41.30,41.298255,41.30,16.23,0.01245,392.892120,0.514185,001,...,0.0,7.513687,1.421085e-14,0.000000,2.250799,0.161524,29.798394,41.298255,7.344123,"MULTIPOLYGON (((303353.460 6552989.330, 303341..."


In [4]:
# Build network graph from adjacency matrix
reg_df = reg_gdf[["regine", "regine_down", "a_cat_land_km2"]]
g = teo.model.build_graph(reg_df, id_col="regine", next_down_col="regine_down")

In [8]:
g1 = teo.vis.plot_network(
    g,
    "001_023",
    direct="up",
    stat="local",
    quant="a_cat_land_km2",
    image_path="test.png",
)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.544149 to fit


In [6]:
# def draw_graph3(
#     networkx_graph,
#     notebook=True,
#     output_filename="graph.html",
#     show_buttons=False,
#     only_physics_buttons=False,
# ):
#     """
#             This function accepts a networkx graph object,
#             converts it to a pyvis network object preserving its node and edge attributes,
#             and both returns and saves a dynamic network visualization.

#             Valid node attributes include:
#                 "size", "value", "title", "x", "y", "label", "color".

#                 (For more info: https://pyvis.readthedocs.io/en/latest/documentation.html#pyvis.network.Network.add_node)

#             Valid edge attributes include:
#                 "arrowStrikethrough", "hidden", "physics", "title", "value", "width"

#                 (For more info: https://pyvis.readthedocs.io/en/latest/documentation.html#pyvis.network.Network.add_edge)


#             Args:
#                 networkx_graph: The graph to convert and display
#             notebook: Display in Jupyter?
#                 output_filename: Where to save the converted network
#             show_buttons: Show buttons in saved version of network?
#             only_physics_buttons: Show only buttons controlling physics of network?
#     """

#     # import
#     from pyvis import network as net

#     # make a pyvis network
#     pyvis_graph = net.Network(notebook=notebook)
#     pyvis_graph.width = "1000px"
#     # for each node and its attributes in the networkx graph
#     for node, node_attrs in networkx_graph.nodes(data=True):
#         pyvis_graph.add_node(node, **node_attrs)
#     #         print(node,node_attrs)

#     # for each edge and its attributes in the networkx graph
#     for source, target, edge_attrs in networkx_graph.edges(data=True):
#         # if value/width not specified directly, and weight is specified, set 'value' to 'weight'
#         if (
#             not "value" in edge_attrs
#             and not "width" in edge_attrs
#             and "weight" in edge_attrs
#         ):
#             # place at key 'value' the weight of the edge
#             edge_attrs["value"] = edge_attrs["weight"]
#         # add the edge
#         pyvis_graph.add_edge(source, target, **edge_attrs)

#     # turn buttons on
#     if show_buttons:
#         if only_physics_buttons:
#             pyvis_graph.show_buttons(filter_=["physics"])
#         else:
#             pyvis_graph.show_buttons()

#     # return and also save
#     return pyvis_graph.show(output_filename)

In [7]:
# draw_graph3(
#     g,
#     notebook=False,
#     output_filename="graph.html",
#     show_buttons=False,
#     only_physics_buttons=False,
# )